In [1]:
import gym_games
import random
import gymnasium as gym
import numpy as np
from DeepQResNet import DQN

env = gym.make('Othello-v0',render_mode='human')
state_shape = (8, 8, 1)  # Adding the channel dimension
Blackdqn = DQN(state_shape,env.action_space.n)
Whitedqn = DQN(state_shape,env.action_space.n)

def GetPolicy(turn,obs,actions):
    if(turn==1):
        return Blackdqn.BehaviorPolicy(obs,actions)
    else:
        return Whitedqn.BehaviorPolicy(obs,actions)
def InsertBuffer(turn,oldobs, action, reward,obs,done,actions):
    if(turn==1):
        Blackdqn.InsertBuffer(oldobs, action, reward,obs,done,actions)
    else:
        Whitedqn.InsertBuffer(oldobs, action, reward,obs,done,actions)

In [2]:
from DeepQResNet import DQN
import random
env.metadata['autoplay'] = True
env.metadata['render_fps'] = 1500000
obs, reward, done, _, info = env.reset()

BlackmodelCount = 0
WhitemodelCount = 0
for episode in range(1,10000):
    done = False
    obs, reward, done, _, info = env.reset()
    steps = 0
    maxiter = 64
    while (not done) and (steps<maxiter):
        actions = info['action']
        turn = info['turn']
        if actions:
            oldobs = obs
            action = GetPolicy(turn,obs,actions)
            obs, reward, done, _, info = env.step(action)
            InsertBuffer(turn,oldobs, action, reward,obs,done,actions)
            steps += 1
    # 학습 및 손실 값 저장
    blakloss = Blackdqn.train()
    whiteloss = Whitedqn.train()
    if blakloss is not None:
        print(f"Episode {episode + 1}, BlackLoss: {blakloss}\n")
        Blackdqn.update_target_model()
    if whiteloss is not None:
        print(f"Episode {episode + 1}, WhiteLoss: {whiteloss}\n")
        Whitedqn.update_target_model()
    if(episode % 50 == 0):
        print('save ')
        Whitedqn.save(f"model/WhiteModel_{WhitemodelCount}.weights.h5")
        Blackdqn.save(f"model/BlackModel_{BlackmodelCount}.weights.h5")
        BlackmodelCount+=1
        WhitemodelCount+=1

c:\Users\zx288\anaconda3\envs\Pytorch\Lib\site-packages\gymnasium\utils\passive_env_checker.py:193: UserWarning: WARN: The result returned by `env.reset()` should be `(obs, info)` by default, , where `obs` is a observation and `info` is a dictionary containing additional information.
  logger.warn(


Episode 6, BlackLoss: 4.072630405426025

Episode 6, WhiteLoss: 4.445772171020508

Episode 10, BlackLoss: 4.755912780761719

Episode 10, WhiteLoss: 4.188843727111816

Episode 14, BlackLoss: 3.7442519664764404

Episode 15, WhiteLoss: 5.174096584320068

Episode 19, BlackLoss: 3.2047219276428223

Episode 19, WhiteLoss: 5.274001121520996

Episode 23, BlackLoss: 2.7730603218078613

Episode 23, WhiteLoss: 5.293902397155762

Episode 27, BlackLoss: 2.445937395095825

Episode 27, WhiteLoss: 5.260217666625977

Episode 31, BlackLoss: 2.6685657501220703

Episode 32, WhiteLoss: 4.876225471496582

Episode 36, BlackLoss: 2.5735602378845215

Episode 36, WhiteLoss: 4.88696813583374

Episode 40, BlackLoss: 2.417860269546509

Episode 40, WhiteLoss: 4.620471477508545

Episode 44, BlackLoss: 2.4135396480560303

Episode 45, WhiteLoss: 4.649406433105469

Episode 49, BlackLoss: 2.3963422775268555

Episode 49, WhiteLoss: 4.451260566711426

save 
Episode 53, BlackLoss: 2.414201498031616

Episode 53, WhiteLoss: 4

KeyboardInterrupt: 

In [22]:
import random
def Play(selfPlayer,version,r):
    Blackdqn.load( f"model/BlackModel_{version}.weights.h5")
    Whitedqn.load(f"model/WhiteModel_{version}.weights.h5")
    obs, reward, done, _, info = env.reset()
    print(info)
    if(selfPlayer==1):
        env.metadata['autoplay'] = False
    else:
        env.metadata['autoplay'] = True
    while not done:
        if(info['turn']==1):
            if(selfPlayer==1):
                obs,reward,done,_,info = env.step(None)
            else:
                action = env.get_valid_actions()
                if(r):
                    action = random.choice(action)
                else:
                    action = Blackdqn.EstimatePolicy(obs,action)
                obs,reward,done,_,info = env.step(action)
            if(info['turn']!=1):
                env.metadata['autoplay'] =  not env.metadata['autoplay']
                print("Black",reward, done)
                env.render()
        else:
            if(selfPlayer==2):
                obs,reward,done,_,info = env.step(None)
            else:
                action = env.get_valid_actions()
                if(r):
                    action = random.choice(action)
                else:
                    action = Whitedqn.EstimatePolicy(obs,action)
                obs,reward,done,_,info = env.step(action)
            if(info['turn'] !=2):
                env.metadata['autoplay'] = not env.metadata['autoplay']
                print("white",reward, done)
                env.render()

Play(1,11,False)

{'autoplay': True, 'turn': 1, 'action': [20, 29, 34, 43], 'blackSum': 2, 'whiteSum': 2}
Black 3 False
white 0 False


c:\Users\zx288\anaconda3\envs\Pytorch\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_valid_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_valid_actions` for environment variables or `env.get_wrapper_attr('get_valid_actions')` that will search the reminding wrappers.
  logger.warn(


Black 3 False
white 0 False
Black 7 False
white 0 False
Black 3 False
white 0 False
Black -1 False
white 0 False
Black 1 False
white 0 False
Black -1 False
white 0 False
Black -1 False
white 0 False
Black 3 False
white 0 False
Black 3 False
white 0 False
Black 3 False
white 0 False
Black 3 False
white 0 False
Black -1 False
white 0 False
Black 3 False
white 0 False


KeyboardInterrupt: 

: 

In [1]:
env.close()

NameError: name 'env' is not defined